# Defined Origin–Destination Pairs Tutorial
This tutorial demonstrates how to run a **Defined Origin–Destination (OD) analysis** with RA2CE.
You explicitly provide origins and destinations, and RA2CE calculates shortest or quickest routes between them.

If you are not yet familiar with preparing origins and destinations shapefiles, see the 
[Origins and Destinations Data Preparation](../tutorials/accessibility.prepare_data_origin_destinations.html) tutorial.

## Step 1: Import Libraries and Set Paths

In [1]:
from pathlib import Path

from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses, AnalysisConfigData
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import AnalysisLossesEnum
from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
from ra2ce.network import RoadTypeEnum
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (
    NetworkSection, NetworkConfigData, OriginsDestinationsSection, HazardSection, CleanupSection
)
from ra2ce.ra2ce_handler import Ra2ceHandler

# Specify the path to your RA2CE project folder and input data
root_dir = Path("data", "pairs_origins_destinations")
network_path = root_dir.joinpath('static', 'network')

c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 2: Define Network with Origins & Destinations
Define the network configuration, in this case downloaded from OSM and clipped to a region polygon. Only specific road types are included.

In [2]:
network_section = NetworkSection(
    source=SourceEnum.OSM_DOWNLOAD,
    polygon=network_path.joinpath("region_polygon.geojson"),
    network_type=NetworkTypeEnum.DRIVE,
    road_types=[
        RoadTypeEnum.MOTORWAY,
        RoadTypeEnum.MOTORWAY_LINK,
        RoadTypeEnum.PRIMARY,
        RoadTypeEnum.PRIMARY_LINK,
        RoadTypeEnum.SECONDARY,
        RoadTypeEnum.SECONDARY_LINK,
        RoadTypeEnum.TERTIARY,
        RoadTypeEnum.TERTIARY_LINK,
        RoadTypeEnum.RESIDENTIAL,
    ],
    save_gpkg=True
)

Define origins and destinations shapefiles using [OriginsDestinationsSection](../api/ra2ce.network.network_config_data.html#ra2ce.network.network_config_data.network_config_data.OriginsDestinationsSection){.api-ref}. Specify the shortcut names and attributes in the shapefiles.

In [4]:
origin_destination_section = OriginsDestinationsSection(
    origins=network_path.joinpath("origins.shp"),
    destinations=network_path.joinpath("destinations.shp"),
    origins_names="A",
    destinations_names="B",
    origin_count="POPULATION",
)

In [5]:
network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=root_dir.joinpath('static'),
    network=network_section,
    origins_destinations=origin_destination_section,
)

## Step 3: Define the Analysis
Specify the analysis type as [AnalysisLossesEnum.OPTIMAL_ROUTE_ORIGIN_DESTINATION](../api/ra2ce.analysis.analysis_config_data.enums.html#module-ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum){.api-ref}, which calculates optimal routes for the OD pairs.

In [6]:
analyse_section = AnalysisSectionLosses(
    name="origin_destination_without_hazard",
    analysis=AnalysisLossesEnum.OPTIMAL_ROUTE_ORIGIN_DESTINATION,
    weighing=WeighingEnum.LENGTH,
    calculate_route_without_disruption=True,
    save_csv=True,
    save_gpkg=True,
)

analysis_config_data = AnalysisConfigData(
    output_path=root_dir.joinpath("output"),
    static_path=root_dir.joinpath('static'),
    analyses=[analyse_section],
)

## Step 4: Run the Analysis
Use [Ra2ceHandler](../api/ra2ce.handler.html#ra2ce.ra2ce_handler.Ra2ceHandler){.api-ref} to configure and run the analysis.

In [7]:
handler = Ra2ceHandler.from_config(
    network=network_config_data,
    analysis=analysis_config_data
)
handler.configure()
handler.run_analysis()

c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\osmnx\simplification.py:513: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = convert.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\hauth\miniforge3\envs\ra2ce_env\Lib\site-packages\osmnx\simplification.py:560: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid
100%|██████████| 4182/4182 [00:00<00:00, 256955.88it/s]
2025-10-01 05:03:11 PM - [avg_speed_calculator.py:175] - root - WARNING - No valid file found with average speeds data\pairs_origins_destinations\static\output_graph\avg_speed.csv, calculating and saving them instead.
2025-10-01 05:03:11 PM - [avg_speed_calculator.py:150]

[AnalysisResultWrapper(results_collection=[AnalysisResult(analysis_result=          o_node       d_node origin destination  \
 0    12233733592   5632424987    A_0         B_0   
 1    12233733592  12233733648    A_0         B_1   
 2    12233733592   1934244652    A_0         B_2   
 3    12233733592  12233733649    A_0         B_3   
 4    12233733592  12233733650    A_0         B_4   
 ..           ...          ...    ...         ...   
 681  12233733647   1934244652  A_109         B_2   
 682  12233733647  12233733649  A_109         B_3   
 683  12233733647  12233733650  A_109         B_4   
 684  12233733647  12233733651  A_109         B_5   
 685  12233733647  12233733651  A_109         B_6   
 
                                               opt_path    length  \
 0    [12233733592, 1833347000, 9743549768, 19347726...  5247.890   
 1    [12233733592, 1833347000, 9743549768, 19347726...  5274.890   
 2    [12233733592, 1833347000, 9743549768, 19347726...  2541.245   
 3    [122337

## Step 5: Inspect Results
Results are stored in the `output` folder and include both CSV and GeoPackage (GPKG) files. You can open the GPKG in GIS software or load it in Python using GeoPandas.

In [ ]:
import geopandas as gpd

results_gpkg = root_dir.joinpath(
    "output", 
    "optimal_route_origin_destination",  
    "origin_destination_without_hazard.gpkg"
)
gdf = gpd.read_file(results_gpkg)
gdf.head()

![Accessability of population to health centers]( /_resources/Beira_OD_no_hazard.png )
All shortest routes from a chosen origin (red circle) to all possible destinations (green stars).